<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/trac2engtwtrobertabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


     |████████████████████████████████| 2.8 MB 5.7 MB/s 
     |████████████████████████████████| 311 kB 10.1 MB/s 
     |████████████████████████████████| 3.3 MB 19.2 MB/s 
     |████████████████████████████████| 596 kB 28.3 MB/s 
     |████████████████████████████████| 895 kB 29.5 MB/s 
     |████████████████████████████████| 67 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 38.6 MB/s 
     |████████████████████████████████| 1.1 MB 33.7 MB/s 
     |████████████████████████████████| 133 kB 45.5 MB/s 
     |████████████████████████████████| 243 kB 47.3 MB/s 
     |████████████████████████████████| 271 kB 47.1 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 49.1 MB/s 


In [ ]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [ ]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 126 µs (started: 2022-02-22 05:27:52 +00:00)


In [ ]:
import torch
torch.cuda.empty_cache()


time: 927 ms (started: 2022-02-22 05:27:52 +00:00)


In [ ]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 187 ms (started: 2022-02-22 05:27:53 +00:00)


In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 2.74 s (started: 2022-02-22 05:27:53 +00:00)


In [ ]:
import pandas as pd
import numpy as np


time: 1.63 ms (started: 2022-02-22 05:27:56 +00:00)


In [ ]:
from transformers import DataCollatorWithPadding


time: 255 ms (started: 2022-02-22 05:27:56 +00:00)


In [ ]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "rohanrajpal/bert-base-en-hi-codemix-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

time: 3.52 s (started: 2022-02-22 05:27:59 +00:00)


In [ ]:
train= pd.read_csv('/content/agr_hi_train.csv', header= None, sep=',')

In [ ]:
train.shape


(12000, 3)

In [ ]:
train.head() 


,id,text,label
0,facebook_corpus_msr_401470,Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu...,OAG
1,facebook_corpus_msr_386695,Bhai 60sal pehle desh me kya tha pehle pta kro...,CAG
2,facebook_corpus_msr_373389,chutiya friday ko isliye releae krte kyoki wee...,CAG
3,facebook_corpus_msr_917635,जय मोदीराज,CAG
4,facebook_corpus_msr_382517,UPA walo ne bahot kuch kr diya tha desh k liye,CAG


In [ ]:
import pandas as pd

In [ ]:
train.columns=['id','text','label']


In [ ]:
train['label'].value_counts()


1    9725
0    2275
Name: label, dtype: int64

In [ ]:
train.loc[(train.label == 'NAG'), 'label'] = 0
train.loc[(train.label == 'CAG'), 'label'] = 1
train.loc[(train.label == 'OAG'), 'label'] = 1

In [ ]:
dev= pd.read_csv('/content/agr_hi_dev.csv',header=None)


time: 42.5 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
dev.columns=['id','text','label']


time: 3.82 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
dev

,id,text,label
0,facebook_corpus_msr_333008,RANDTV tumhare najayaz baap is area hai ki wah...,OAG
1,facebook_corpus_msr_690306,First stage par dus jootey khaye Grover se,OAG
2,facebook_corpus_msr_382520,Salman aur aamir ki kounsi movie release huyee...,OAG
3,facebook_corpus_msr_401748,Wk aur bhakt aa gya. Abe americans ka bolna h...,CAG
4,facebook_corpus_msr_656339,Beta to tu apne baap ka hai ...permission to t...,NAG
...,...,...,...
2996,facebook_corpus_msr_356273,Agar sarkar ne ab bhi kush nahi kia to ak din ...,CAG
2997,facebook_corpus_msr_349781,kuch bhi nahi kiya par ek kaam kiyaa vo ki sab...,CAG
2998,facebook_corpus_msr_1906980,Yaar isliye to jansatta UNFOLLOW kiya tha. Ab ...,CAG
2999,facebook_corpus_msr_1391645,ye log na to janneta kahlane layak hai na hi i...,OAG


time: 32.8 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
dev.loc[(dev.label == 'NAG'), 'label'] = 0
dev.loc[(dev.label == 'CAG'), 'label'] = 1
dev.loc[(dev.label == 'OAG'), 'label'] = 1

time: 7.13 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
import re

time: 1.02 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

time: 6.8 ms (started: 2022-02-22 05:28:03 +00:00)


In [ ]:
X_train=train.text.values
X_dev=dev.text.values
y_train=train.label.values
y_dev=dev.label.values

time: 3.49 ms (started: 2022-02-22 05:28:42 +00:00)


In [ ]:
train['text'].isnull().sum()


0

time: 6.74 ms (started: 2022-02-22 05:28:45 +00:00)


In [ ]:
train=train.dropna()


time: 16.4 ms (started: 2022-02-22 05:28:39 +00:00)


In [ ]:
train_texts= np.array([preprocess(text) for text in X_train])
dev_texts= np.array([preprocess(text) for text in X_dev])
num_labels=2


time: 375 ms (started: 2022-02-22 05:28:49 +00:00)


In [ ]:
train_texts=train_texts.tolist()
dev_texts=dev_texts.tolist()

time: 50.2 ms (started: 2022-02-22 05:28:52 +00:00)


In [ ]:
train_texts[0:20]

['Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu 😂 1 hazar saal tjhy ghulam bana kr rkha tha bhol gaya tu',
 'Bhai 60sal pehle desh me kya tha pehle pta kro desh waise ka waise kaise hai',
 'chutiya friday ko isliye releae krte kyoki week end ke 3 din milte h to log free hokr film dekh sake..!! hd h yrr chutiyapa ki',
 'जय मोदीराज',
 'UPA walo ne bahot kuch kr diya tha desh k liye',
 'Pan ko Aadhar se link ki zarurat kuy hai? Supreme Court ne ye bola hi nahi',
 'काकर पाथर जोड़ के मस्जिद लई बनाय।\n\nता चढ़ि मुल्ला बांघ दे , क्या बहरा हुआ खुदाय। \n\nयह मैं नहीं कबीर ने कहा था।',
 'Us raat tere papa k jageh mera sperm gya tha use of technology \nAur tere papa ko pta v na chala 😂😂😂\nChutiya',
 'गटर के कीड़े',
 'Waise bandhu jet lag se bachne ke liye Raat ko 10 baje ke baad so jao',
 'get your love back +91-(9815215009) and #all_problems_solution_with_in_72_hours one call change life \nसमस्या चाहे कैसी भी हो जड़ से खत्म।\nबड़ी से बड़ी समस्या का समाधान।101%\nअपने मन चाहे प्यार को पाना, काम कारोबा

time: 4.29 ms (started: 2022-02-22 05:28:55 +00:00)


In [ ]:
train_encodings = tokenizer(train_texts,truncation=True,)
dev_encodings = tokenizer(dev_texts,truncation=True)

time: 2.06 s (started: 2022-02-22 05:28:58 +00:00)


In [ ]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 8.82 ms (started: 2022-02-22 05:29:03 +00:00)


In [ ]:
train_dataset = TaskDatasetB(train_encodings,y_train)
dev_dataset = TaskDatasetB(dev_encodings,y_dev)

time: 4.82 ms (started: 2022-02-22 05:29:06 +00:00)


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 1.74 ms (started: 2022-02-22 05:29:09 +00:00)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'hin-trac1_fin',
    run_name = "hi-2epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 2,
    load_best_model_at_end = True,
    push_to_hub='hin-trac1_fin'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


time: 15.2 ms (started: 2022-02-22 05:30:40 +00:00)


In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 9.3 ms (started: 2022-02-22 05:30:43 +00:00)


In [ ]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2,ignore_mismatched_sizes=True)


Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rohanrajpal/bert-base-en-hi-codemix-cased and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 29.9 s (started: 2022-02-22 05:29:26 +00:00)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/hin-trac1_fin into local empty directory.


time: 8.08 s (started: 2022-02-22 05:30:47 +00:00)


In [ ]:
trainer.train()


***** Running training *****
  Num examples = 11999
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3000


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,0.415100,0.356809,0.868044,0.857391,0.868044,0.850140
2,0.344900,0.375760,0.874375,0.867331,0.874375,0.856319


***** Running Evaluation *****
  Num examples = 3001
  Batch size = 8
Saving model checkpoint to hin-trac1_fin/checkpoint-1500
Configuration saved in hin-trac1_fin/checkpoint-1500/config.json
Model weights saved in hin-trac1_fin/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3001
  Batch size = 8
Saving model checkpoint to hin-trac1_fin/checkpoint-3000
Configuration saved in hin-trac1_fin/checkpoint-3000/config.json
Model weights saved in hin-trac1_fin/checkpoint-3000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from hin-trac1_fin/checkpoint-1500 (score: 0.35680854320526123).


TrainOutput(global_step=3000, training_loss=0.39668375651041665, metrics={'train_runtime': 1791.5185, 'train_samples_per_second': 13.395, 'train_steps_per_second': 1.675, 'total_flos': 2017023739275600.0, 'train_loss': 0.39668375651041665, 'epoch': 2.0})

time: 29min 51s (started: 2022-02-22 05:31:06 +00:00)


In [ ]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 3001
  Batch size = 8


{'epoch': 2.0,
 'eval_acc': 0.8680439853382206,
 'eval_f1': 0.850139786491252,
 'eval_loss': 0.35680854320526123,
 'eval_precision': 0.8573911510505229,
 'eval_recall': 0.8680439853382206,
 'eval_runtime': 66.3869,
 'eval_samples_per_second': 45.205,
 'eval_steps_per_second': 5.664}

time: 1min 6s (started: 2022-02-22 06:00:58 +00:00)


In [ ]:
import torch
torch.cuda.empty_cache()

time: 642 ms (started: 2022-02-22 06:02:04 +00:00)


In [ ]:
model_path = 'hin-trac1_fin',



time: 810 µs (started: 2022-02-22 06:02:05 +00:00)


In [ ]:
model.push_to_hub('hin-trac1_fin')


Configuration saved in hin-trac1_fin/config.json
Model weights saved in hin-trac1_fin/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.39k/679M [00:00<?, ?B/s]

Upload file runs/Feb22_05-30-40_71be2d3e7e81/events.out.tfevents.1645509724.71be2d3e7e81.86.2: 100%|##########…

Upload file runs/Feb22_05-30-40_71be2d3e7e81/1645507866.9120207/events.out.tfevents.1645507866.71be2d3e7e81.86…

Upload file runs/Feb22_05-30-40_71be2d3e7e81/events.out.tfevents.1645507866.71be2d3e7e81.86.0:  64%|######4   …

To https://huggingface.co/Maha/hin-trac1_fin
   c933c9f..fd39670  main -> main



'https://huggingface.co/Maha/hin-trac1_fin/commit/fd396708064679c232edd6b5388d497520f9d952'

time: 11min 28s (started: 2022-02-22 06:02:05 +00:00)


In [ ]:
tokenizer.push_to_hub('hin-trac1_fin')


tokenizer config file saved in hin-trac1_fin/tokenizer_config.json
Special tokens file saved in hin-trac1_fin/special_tokens_map.json
To https://huggingface.co/Maha/hin-trac1_fin
   fd39670..04d02a5  main -> main



'https://huggingface.co/Maha/hin-trac1_fin/commit/04d02a59844416ce825a7cf9d19b8207668fd37e'

time: 6.42 s (started: 2022-02-22 06:13:34 +00:00)
